# Dataset Exploration

In [2]:
import json

import pandas as pd

In [27]:
data_dir = 'data/'

# Load the data
df_games = pd.read_csv(data_dir + 'games.csv')
df_recommendations = pd.read_csv(data_dir + 'recommendations.csv')
df_users = pd.read_csv(data_dir + 'users.csv')

with open(data_dir + 'games_metadata.json', 'r') as f:
    metadata = json.load(f)

df_metadata = pd.DataFrame(metadata)

## Games

In [4]:
df_games.head()

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True
1,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,2.99,0.0,True
2,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True
3,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,14.99,0.0,True
4,249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,11.99,0.0,True


### Games Completeness Check

In [34]:
print("=== Games DataFrame Analysis ===")
print("\nTotal number of games:", len(df_games))
print("\nMissing values in each column:")
print(df_games.isnull().sum())

# Drop price_final column as it's time-sensitive, and we don't care about it here
df_games = df_games.drop('price_final', axis=1)

# Platform availability check
print("\nPlatform availability:")
for col in ['win', 'mac', 'linux', 'steam_deck']:
    print(f"Games available on {col}: {len(df_games[df_games[col] == 1])} ({(len(df_games[df_games[col] == 1])/len(df_games)*100):.2f}%)")

# Games with price_original = 0 might be legitimate free content
free_games = len(df_games[df_games['price_original'] == 0])
print(f"\nFree games/content: {free_games} ({(free_games/len(df_games)*100):.2f}%)")

=== Games DataFrame Analysis ===

Total number of games: 50872

Missing values in each column:
app_id            0
title             0
date_release      0
win               0
mac               0
linux             0
rating            0
positive_ratio    0
user_reviews      0
price_final       0
price_original    0
discount          0
steam_deck        0
dtype: int64

Platform availability:
Games available on win: 50076 (98.44%)
Games available on mac: 13018 (25.59%)
Games available on linux: 9041 (17.77%)
Games available on steam_deck: 50870 (100.00%)

Free games/content: 10090 (19.83%)


## Recommendations

In [5]:
df_recommendations.head()

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id
0,975370,0,0,2022-12-12,True,36.3,51580,0
1,304390,4,0,2017-02-17,False,11.5,2586,1
2,1085660,2,0,2019-11-17,True,336.5,253880,2
3,703080,0,0,2022-09-23,True,27.4,259432,3
4,526870,0,0,2021-01-10,True,7.9,23869,4


### Recommendations Completeness Check

In [32]:
print("\n=== Recommendations DataFrame Analysis ===")
print("\nTotal number of recommendations:", len(df_recommendations))
print("\nMissing values in each column:")
print(df_recommendations.isna().sum())

print("\nPotentially interesting values:")
zero_hours = len(df_recommendations[df_recommendations['hours'] == 0])
print(f"Reviews with zero hours played: {zero_hours} ({(zero_hours / len(df_recommendations) * 100):.2f}%)")
print(f"Not recommended games: {len(df_recommendations[df_recommendations['is_recommended'] == False])} ({(len(df_recommendations[df_recommendations['is_recommended'] == False]) / len(df_recommendations) * 100):.2f}%)")



=== Recommendations DataFrame Analysis ===

Total number of recommendations: 41154794

Missing values in each column:
app_id            0
helpful           0
funny             0
date              0
is_recommended    0
hours             0
user_id           0
review_id         0
dtype: int64

Potentially interesting values:
Reviews with zero hours played: 176329 (0.43%)
Not recommended games: 5850396 (14.22%)


## Users

In [6]:
df_users.head()

,user_id,products,reviews
0,7360263,359,0
1,14020781,156,1
2,8762579,329,4
3,4820647,176,4
4,5167327,98,2


### Users Completeness Check

In [35]:
print("\n=== Users DataFrame Analysis ===")
print("\nTotal number of users:", len(df_users))
print("\nMissing values in each column:")
print(df_users.isna().sum())

print("\nUsers with zero values:")
for col in ['products', 'reviews']:
    zero_count = len(df_users[df_users[col] == 0])
    print(f"Users with zero {col}: {zero_count} ({(zero_count / len(df_users) * 100):.2f}%)")


=== Users DataFrame Analysis ===

Total number of users: 14306064

Missing values in each column:
user_id     0
products    0
reviews     0
dtype: int64

Users with zero values:
Users with zero products: 139318 (0.97%)
Users with zero reviews: 525005 (3.67%)


## Games metadata

In [22]:
df_metadata.head()

,app_id,description,tags
0,13500,Enter the dark underworld of Prince of Persia ...,"[Action, Adventure, Parkour, Third Person, Gre..."
1,22364,,[Action]
2,113020,Monaco: What's Yours Is Mine is a single playe...,"[Co-op, Stealth, Indie, Heist, Local Co-Op, St..."
3,226560,Escape Dead Island is a Survival-Mystery adven...,"[Zombies, Adventure, Survival, Action, Third P..."
4,249050,Dungeon of the Endless is a Rogue-Like Dungeon...,"[Roguelike, Strategy, Tower Defense, Pixel Gra..."


### Metadata Completeness Check

In [25]:
#####################
# Tags distribution #
#####################

# Explode (Transform each element of a list-like to a row) the tags list into separate rows
tags_df = df_metadata[['app_id', 'tags']].explode('tags')

print("\nMost common tags:")
print(tags_df['tags'].value_counts())

# Percentage of games with each tag
total_games = len(df_metadata)
tag_percentages = (tags_df['tags'].value_counts() / total_games * 100).round(2)

print("\nPercentage of games with each tag:")
print(tag_percentages)

# Statistics about number of tags per game
tags_per_game = df_metadata['tags'].str.len()

print("\nTags per game statistics:")
print(tags_per_game.describe())


Most common tags:
tags
Indie            27957
Singleplayer     22566
Action           21897
Adventure        20183
Casual           17844
                 ...  
Volleyball          10
Hobby Sim            6
Tile-Matching        5
Fox                  5
Coding               4
Name: count, Length: 441, dtype: int64

Percentage of games with each tag:
tags
Indie            54.96
Singleplayer     44.36
Action           43.04
Adventure        39.67
Casual           35.08
                 ...  
Volleyball        0.02
Hobby Sim         0.01
Tile-Matching     0.01
Fox               0.01
Coding            0.01
Name: count, Length: 441, dtype: float64

Tags per game statistics:
count    50872.000000
mean        11.484019
std          7.256911
min          0.000000
25%          4.000000
50%         11.000000
75%         20.000000
max         20.000000
Name: tags, dtype: float64


In [17]:
# Check for empty lists in tags
empty_tags = df_metadata[df_metadata['tags'].apply(lambda x: len(x) == 0)]
print("Number of games with empty tags list:", len(empty_tags))

# Check for empty strings in descriptions
empty_descriptions = df_metadata[df_metadata['description'] == '']
print("\nNumber of games with empty descriptions:", len(empty_descriptions))

# Check for both empty tags and empty descriptions
both_empty = df_metadata[(df_metadata['tags'].apply(lambda x: len(x) == 0)) & 
                        (df_metadata['description'] == '')]
print("\nNumber of games with both empty tags and descriptions:", len(both_empty))

# Summary statistics
print("\nDataset summary:")
print(f"Total number of games: {len(df_metadata)}")
print(f"Complete records (non-empty tags and descriptions): {len(df_metadata[(df_metadata['tags'].apply(lambda x: len(x) > 0)) & (df_metadata['description'] != '')])}") 
print(f"Records with either empty tags or empty descriptions: {len(df_metadata[(df_metadata['tags'].apply(lambda x: len(x) == 0)) | (df_metadata['description'] == '')])}")


Number of games with empty tags list: 1244

Number of games with empty descriptions: 10373

Number of games with both empty tags and descriptions: 1229

Dataset summary:
Total number of games: 50872
Complete records (non-empty tags and descriptions): 40484
Records with either empty tags or empty descriptions: 10388


In [31]:
# Data consistency across DataFrames
print("\n=== Cross-DataFrame Consistency ===")
print(f"Users in users df but not in recommendations: {len(set(df_users['user_id']) - set(df_recommendations['user_id']))}")
print(f"Games in games df but not in recommendations: {len(set(df_games['app_id']) - set(df_recommendations['app_id']))}")


=== Cross-DataFrame Consistency ===
Users in users df but not in recommendations: 525005
Games in games df but not in recommendations: 13262


# Handling missing data
There are a few challenges here that we should address:

### Missing Data Issues:
- ~2.4% games have empty tags (1,244 out of 50,872)
- ~20.4% games have empty descriptions (10,373)
- ~2.4% games have both empty tags and descriptions (1,229)
- Only ~79.6% games (40,484) have complete records

### Language Issue:
For descriptions in different languages we could either:
1. Use language detection and filter out non-English descriptions
2. Use a translation service to translate the descriptions to English

### Missing Data:
1. Fetch lacking data from Steam Web API
2. Remove games with missing data

## 1st approach: Filling missing data

This approach requires a Steam API key and a script to fetch the missing data. For this case we implemented a steam_scraper which can be found under a folder with corresponding name. The problem is that, the missing data is missing at the core (Steam platform itself), so it's not possible to fetch it properly.

## 2nd approach: Removing the games with missing data

In [36]:
from langdetect import detect

def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False


complete_metadata = df_metadata[
    (df_metadata['tags'].apply(len) > 0) & 
    (df_metadata['description'] != '') &
    (df_metadata['description'].apply(is_english))
]

print("\nDataset sizes:")
print(f"Original: {len(df_metadata)} games")
print(f"Complete English records: {len(complete_metadata)} games")


Dataset sizes:
Original: 50872 games
Complete English records: 39540 games
